In [ ]:
import HyperbolicDiffEq; #reload("HyperbolicDiffEq")
using OrdinaryDiffEq

using Plots; pyplot()
using LaTeXStrings

HCL=HyperbolicDiffEq
LobattoLegendre = HCL.LobattoLegendre; UniformPeriodicMesh1D = HCL.UniformPeriodicMesh1D
compute_coefficients = HCL.compute_coefficients; evaluate_coefficients = HCL.evaluate_coefficients
semidiscretise = HCL.semidiscretise; UniformPeriodicFluxDiffDisc2D = HCL.UniformPeriodicFluxDiffDisc2D
EnergyConservativeFlux = HCL.EnergyConservativeFlux; GodunovFLux = HCL.godunov
EulerVar2D = HCL.EulerVar2D; conserved_variables = HCL.conserved_variables; SuliciuFlux=HCL.SuliciuFlux
LocalLaxFriedrichsFlux = HCL.LocalLaxFriedrichsFlux; ChandrashekarFluxEC=HCL.ChandrashekarFluxEC
IsmailRoeFluxEC=HCL.IsmailRoeFluxEC; CentralFlux=HCL.CentralFlux; RanochaFluxECandKEP=HCL.RanochaFluxECandKEP
MorinishiFlux=HCL.MorinishiFlux; DucrosEtAlFlux=HCL.DucrosEtAlFlux; KennedyGruberFlux=HCL.KennedyGruberFlux
PirozzoliFlux=HCL.PirozzoliFlux; integrate=HCL.integrate; kinetic_energy=HCL.kinetic_energy; entropy=HCL.entropy
UniformPeriodicFluxDiffDisc3D=HCL.UniformPeriodicFluxDiffDisc3D

const γ = 1.4
balance_law = HCL.Euler{Float64,3}(γ)

ϱ₀(x,y,z)  = 1.
vx₀(x,y,z) = sin(x)*cos(y)*cos(z)
vy₀(x,y,z) = -cos(x)*sin(y)*cos(z)
vz₀(x,y,z) = zero(x)
p₀(x,y,z)  = 100/γ + (cos(2x)*cos(2z)+2*cos(2y)+2*cos(2x)+cos(2y)*cos(2z))/16

u₀(x,y,z) = conserved_variables(ϱ₀(x,y,z), vx₀(x,y,z), vy₀(x,y,z), vz₀(x,y,z), p₀(x,y,z), balance_law)
xmin=0.; xmax=2π; ymin=0.; ymax=2π; zmin=0.; zmax=2π

#fvol = CentralFlux()
#fvol = MorinishiFlux()
#fvol = DucrosEtAlFlux()
#fvol = KennedyGruberFlux()
#fvol = PirozzoliFlux()
#fvol = EnergyConservativeFlux()
#fvol = ChandrashekarFluxEC()
#fvol = IsmailRoeFluxEC()
fvol = RanochaFluxECandKEP()

fnum = fvol
#fnum = SuliciuFlux()
#fnum = LocalLaxFriedrichsFlux()

p = 1
Nx = 5
Ny = 5
Nz = 5

meshx = UniformPeriodicMesh1D(xmin, xmax, Nx)
meshy = UniformPeriodicMesh1D(ymin, ymax, Ny)
meshz = UniformPeriodicMesh1D(zmin, zmax, Nz)
basis = LobattoLegendre(p)

tspan = (0., 1.)
semidisc = UniformPeriodicFluxDiffDisc3D(balance_law, meshx, meshy, meshz, basis, fvol, fnum, true)
ode = semidiscretise(semidisc, u₀, tspan)

#solve(ode, SSPRK104(), dt=tspan[end]-tspan[1], save_everystep=false)
@time sol = solve(ode, SSPRK104(), dt=1/((2p+1)*max(Nx,Ny,Nz)), save_everystep=true);


In [ ]:
idx2ekin(idx) = integrate(u->kinetic_energy(u,balance_law), sol[idx], meshx, meshy, meshz, basis)
idx2entr(idx) = integrate(u->entropy(u,balance_law), sol[idx], meshx, meshy, meshz, basis)

ekin1 = idx2ekin(1)
entr1 = idx2entr(1)

fig_ekin = plot(sol.t, (map(idx2ekin, eachindex(sol))-ekin1)/abs(ekin1), label="", xguide=L"t", yguide="Rel. Kinetic Energy Difference")
fig_entr = plot(sol.t, (map(idx2entr, eachindex(sol))-entr1)/abs(entr1), label="", xguide=L"t", yguide="Rel. Entropy Difference")

plot(fig_ekin, fig_entr)